In [ ]:
import set_path

import numpy as np
import laspy
import geopandas as gpd
from shapely.geometry import Point, Polygon, box
import pandas as pd

from upcp.utils import ahn_utils, clip_utils
from upcp.utils.interpolation import FastGridInterpolator

from gvl.helper_functions import color_clusters
from gvl.tree_detector import DetectorTree
from gvl.ahn_utils import GeoTIFFReader2

In [ ]:
import warnings  # temporary, to supress deprecationwarnings from shapely
warnings.filterwarnings('ignore')

In [ ]:
RD_CRS = 'epsg:28992'
LL_CRS = 'WGS84'

# AHN classification
AHN_OTHER = 1
AHN_GROUND = 2
AHN_BUILDING = 6
AHN_WATER = 9
AHN_ARTIFACT = 26

### TODO make geotiles scraper
We use 5m resolution. The ground filter improvements when using is 0.5m is negligible and running time increases.

In [ ]:
# Use AHN subtiles
tilecode = '25GN1_08'

las_file = f'../datasets/ahn_laz/{tilecode}.LAZ'
tree_ref_file = '../datasets/validation/joined_trees_bgt_gissib_1_5_amsterdam.shp'
ahn_geotiff_folder = '../datasets/ahn_dtm/'

ahn_reader = GeoTIFFReader2(ahn_geotiff_folder, fill_gaps=False,
                            smoothen=True, smooth_thickness=2)

tree_gdf = gpd.read_file(tree_ref_file, crs=RD_CRS)
tree_gdf = tree_gdf.to_crs(RD_CRS) # TODO needed?

In [ ]:
las = laspy.read(las_file)
points_3d = np.vstack((las.x, las.y, las.z)).T
points_rgb = np.vstack((las.red, las.green, las.blue)).T

In [ ]:
pc_header = las.header
offset = 20 # Necesarry to avoid incorrect DTM + LAZ overlap
bbox_original = pc_header.min[0], pc_header.min[1], pc_header.max[0], pc_header.max[1]
bbox = pc_header.min[0]+offset, pc_header.min[1]+offset, pc_header.max[0]-offset, pc_header.max[1]-offset

### Filter 1: scalar fields and clip
- classification -> 'overig label' points

We dont want to filter in the number_of_returns scalar field. It will remove too much valuable tree points. 

In [ ]:
area_mask = clip_utils.rectangle_clip(points_3d, bbox)
class_mask = las.classification == AHN_OTHER

mask = class_mask & area_mask

In [ ]:
gnd_tile = ahn_reader.filter_area(bbox)

fast_z = FastGridInterpolator(
            gnd_tile['x'], gnd_tile['y'], gnd_tile['ground_surface'])
gnd_z = fast_z(points_3d[mask])

In [ ]:
np.count_nonzero(gnd_z == np.nan)

### Filter 2: points close to ground
TODO move this before the KDTree code

In [ ]:
above_ground_in_meters = 2.5
height_mask = points_3d[mask, 2] - gnd_z >= above_ground_in_meters
ground_z = gnd_z[height_mask] # TODO # Ugggly

In [ ]:
points_3d = points_3d[mask][height_mask]
points_rgb = points_rgb[mask][height_mask]
points_i = las.intensity[mask][height_mask]

# LCC

In [ ]:
from upcp.utils.clip_utils import poly_box_clip
from upcp.utils.math_utils import minimum_bounding_rectangle

def _label_tree_like_components(points, ground_z, point_components,
                                tree_points, min_height):
    """ Based on certain properties of a tree we label clusters.  """
    
    tree_mask = np.zeros(len(points), dtype=bool)
    tree_count = 0
    
    if len(tree_points) == 0:
        print('No reference tree points, skipping.')
        return tree_mask

    cc_labels = np.unique(point_components)

    cc_labels = set(cc_labels).difference((-1,))

    for cc in cc_labels:
        # select points that belong to the cluster
        cc_mask = (point_components == cc)

        target_z = ground_z[cc_mask]
        valid_values = target_z[np.isfinite(target_z)]

        if valid_values.size != 0:
            cc_z = np.mean(valid_values)
            min_z = cc_z + min_height
            cluster_height = np.amax(points[cc_mask][:, 2])
            if min_z <= cluster_height:
                mbrect, conv_hull, mbr_width, mbr_length, _ =\
                    minimum_bounding_rectangle(points[cc_mask][:, :2])
                p1 = Polygon(conv_hull)
                for p2 in tree_points:
                    do_overlap = p1.contains(p2)
                    if do_overlap:
                        tree_mask[cc_mask] = True
                        tree_count += 1
                        break

    print(f'{tree_count} tree labelled.')

    return tree_mask

In [ ]:
trees_in_bbox = tree_gdf[tree_gdf.within(box(*bbox, ccw=True))]
tree_points = list(trees_in_bbox['geometry'].values)

In [ ]:
# TODO: progress indication

from upcp.region_growing.label_connected_comp import LabelConnectedComp

grid_size = 0.6
min_component_size = 50
min_height = 3.5

lcc = LabelConnectedComp(grid_size=grid_size,
                         min_component_size=min_component_size)
point_components = lcc.get_components(points_3d)

# Label tree like clusters
tree_mask = _label_tree_like_components(points_3d, ground_z,
                                       point_components,
                                       tree_points, min_height)

In [ ]:
import open3d as o3d

object_pcd = o3d.geometry.PointCloud()
points = np.stack((points_3d[:, 0], points_3d[:, 1], points_3d[:, 2]), axis=-1)
object_pcd.points = o3d.utility.Vector3dVector(points)
object_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=30))
normals = np.matrix.round(np.array(object_pcd.normals), 2)
normals_z = normals[:,2]
normals_of_interest = np.squeeze(np.where(normals_z <= -0.95))
mean_z = np.mean(points_3d[:, 2][normals_of_interest])
normal_indxs_to_change = ((points_3d[:, 2][normals_of_interest] > (mean_z - 2)) & (points_3d[:, 2][normals_of_interest] < (mean_z + 2)))
normals_z[normals_of_interest[normal_indxs_to_change]] = np.absolute(normals_z[normals_of_interest[normal_indxs_to_change]])
normals[:,2] = normals_z

### Noise filter on non tree points and label non trees based on normal values

In [ ]:
grid_size = 0.9
min_component_size = 50

maskk = points_3d[~tree_mask, 0] # Only the possible noise points
mask_ids = np.where(maskk)[0]

lcc = LabelConnectedComp(grid_size=grid_size,
                         min_component_size=min_component_size)
point_components = lcc.get_components(points_3d[mask_ids])

# Noise filter
cc_mask = point_components == -1
print(f'Found {np.count_nonzero(cc_mask)} noise points in '
             + f'clusters <{min_component_size} points.')

label_mask2 = np.zeros((len(points_3d),), dtype=bool)
label_mask2[mask_ids] = cc_mask

# Iterate over the clusters
cc_labels = np.unique(point_components)

cc_labels = set(cc_labels).difference((-1,))

norm_mask = np.zeros(len(normals_z), dtype=bool)

for cc in cc_labels:
    # select points that belong to the cluster
    cc_mask_jm = (point_components == cc)

    if normals_z[mask_ids][cc_mask_jm].mean() > 0.5:
        norm_mask[mask_ids[cc_mask_jm]] = True

In [ ]:
# TODO lelijk met label_mask2
labels = np.zeros((len(points_3d),), dtype='uint16')
labels[~tree_mask] = 0
labels[tree_mask] = 1
labels[label_mask2] = 99
labels[norm_mask] = 55

In [ ]:
header = laspy.LasHeader(point_format=3, version="1.2")
header.offsets = las.header.offsets
header.scales = las.header.scales

new_las = laspy.LasData(header)

new_las.x = points_3d[:, 0]
new_las.y = points_3d[:, 1]
new_las.z = points_3d[:, 2]
new_las.red = points_rgb[:, 0]
new_las.green = points_rgb[:, 1]
new_las.blue = points_rgb[:, 2]
new_las.intensity = points_i

new_las.add_extra_dim(laspy.ExtraBytesParams(name="label", type="uint16",
                                         description="Label"))
new_las.label = labels

new_las.add_extra_dim(laspy.ExtraBytesParams(name="hag", type="uint16",
                                         description="Hag"))
new_las.hag = points_3d[:,2] - ground_z

new_las.add_extra_dim(laspy.ExtraBytesParams(name="normal_x", type="float",
                  description="normal_x"))
new_las.add_extra_dim(laspy.ExtraBytesParams(name="normal_y", type="float",
                  description="normal_y"))    
new_las.add_extra_dim(laspy.ExtraBytesParams(name="normal_z", type="float",
                  description="normal_z"))
new_las.normal_x = normals[:,0]
new_las.normal_y = normals[:,1]
new_las.normal_z = normals[:,2]

new_las.write(f'../datasets/pointcloud/tree_{tilecode}.laz')

### Make test file for RandLA-Net inference and make the seperate ground truth file

In [ ]:
def write_las(prefix, maskje):
    outfile = laspy.create(file_version="1.2", point_format=3)

    outfile.x = points_3d[maskje, 0]
    outfile.y = points_3d[maskje, 1]
    outfile.z = points_3d[maskje, 2]
    outfile.red = points_rgb[maskje, 0]
    outfile.green = points_rgb[maskje, 1]
    outfile.blue = points_rgb[maskje, 2]
    outfile.intensity = points_i[maskje]
    outfile.add_extra_dim(laspy.ExtraBytesParams(name="hag", type="uint16",
                                         description="Hag"))
    outfile.hag = points_3d[maskje,2] - ground_z[maskje]
    
    outfile.add_extra_dim(laspy.ExtraBytesParams(name="normal_x", type="float",
                      description="normal_x"))
    outfile.add_extra_dim(laspy.ExtraBytesParams(name="normal_y", type="float",
                      description="normal_y"))    
    outfile.add_extra_dim(laspy.ExtraBytesParams(name="normal_z", type="float",
                      description="normal_z"))
    outfile.normal_x = normals[maskje,0]
    outfile.normal_y = normals[maskje,1]
    outfile.normal_z = normals[maskje,2]

    outfile.write(f'../datasets/pointcloud/{prefix}_{tilecode}.laz')
    
write_las('ground_truth', labels == 1)
write_las('test', labels == 0)